In [11]:
import requests, json, os, sys, datetime, time
from bs4 import BeautifulSoup
import hjson

In [12]:
url = f"https://delhishelterboard.in/maps/jjc-jsc/jjc-new.php"

In [14]:
s = requests.Session() 
r1= s.get(url)
time.sleep(3)
soup1 = BeautifulSoup( r1.text , 'lxml')

In [15]:
listHolder = soup1.select_one('select')

In [16]:
list1 = []
for optHolder in listHolder.select('option'):
    list1.append({'name': optHolder.text.strip(), 'val': int(optHolder['value'])} )
list1

[{'name': 'JJC Code 1', 'val': 201},
 {'name': 'JJC Code 2', 'val': 202},
 {'name': 'JJC Code 3', 'val': 672},
 {'name': 'JJC Code 4', 'val': 203},
 {'name': 'JJC Code 5', 'val': 38},
 {'name': 'JJC Code 6', 'val': 39},
 {'name': 'JJC Code 7', 'val': 204},
 {'name': 'JJC Code 8', 'val': 41},
 {'name': 'JJC Code 9', 'val': 205},
 {'name': 'JJC Code 10', 'val': 43},
 {'name': 'JJC Code 11', 'val': 44},
 {'name': 'JJC Code 12', 'val': 40},
 {'name': 'JJC Code 13', 'val': 597},
 {'name': 'JJC Code 14', 'val': 206},
 {'name': 'JJC Code 15', 'val': 603},
 {'name': 'JJC Code 16', 'val': 42},
 {'name': 'JJC Code 17', 'val': 604},
 {'name': 'JJC Code 18', 'val': 207},
 {'name': 'JJC Code 19', 'val': 79},
 {'name': 'JJC Code 20', 'val': 80},
 {'name': 'JJC Code 21', 'val': 81},
 {'name': 'JJC Code 22', 'val': 82},
 {'name': 'JJC Code 23', 'val': 87},
 {'name': 'JJC Code 24', 'val': 83},
 {'name': 'JJC Code 25', 'val': 88},
 {'name': 'JJC Code 26', 'val': 84},
 {'name': 'JJC Code 27', 'val': 85},

In [17]:
geojson = {"type": "FeatureCollection",
    "features": []
}

In [25]:
for p in list1[96:]:
    print(p)
    payload=f"jjcid={p['val']}"
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
    }
    r2 = s.post(url, headers=headers, data=payload)
    soup2 = BeautifulSoup( r2.text , 'lxml')
    polyHolder = soup2.select_one('script').contents[0].split('var triangleCoords = ')[1].split(';')[0].replace('\n','')
    polyHolder2 = hjson.loads(polyHolder)
    coords=[]
    for item in polyHolder2:
        lat = item['lat']
        lon = item['lng']
        coords.append([lon,lat])
    
    propHolder1 = soup2.select('div')[1].text.split('\n')
    properties = {}
    for l in propHolder1:
        l1 = l.strip()
        if len(l1):
            l2 = l1.split(' : ')
            properties[l2[0]] = l2[1]
    feature = {"type": "Feature",
        "properties": properties,
        "geometry": {
            "type": "Polygon",
            "coordinates": [coords]
        }
    }
    geojson['features'].append(feature)
    time.sleep(1)

{'name': 'JJC Code 100', 'val': 668}
{'name': 'JJC Code 101', 'val': 216}
{'name': 'JJC Code 102', 'val': 217}
{'name': 'JJC Code 103', 'val': 214}
{'name': 'JJC Code 104', 'val': 215}
{'name': 'JJC Code 105', 'val': 218}
{'name': 'JJC Code 106', 'val': 220}
{'name': 'JJC Code 107', 'val': 219}
{'name': 'JJC Code 108', 'val': 221}
{'name': 'JJC Code 109', 'val': 78}
{'name': 'JJC Code 110', 'val': 75}
{'name': 'JJC Code 111', 'val': 76}
{'name': 'JJC Code 112', 'val': 77}
{'name': 'JJC Code 113', 'val': 607}
{'name': 'JJC Code 114', 'val': 608}
{'name': 'JJC Code 115', 'val': 326}
{'name': 'JJC Code 116', 'val': 316}
{'name': 'JJC Code 117', 'val': 332}
{'name': 'JJC Code 118', 'val': 327}
{'name': 'JJC Code 119', 'val': 312}
{'name': 'JJC Code 120', 'val': 314}
{'name': 'JJC Code 121', 'val': 315}
{'name': 'JJC Code 122', 'val': 328}
{'name': 'JJC Code 123', 'val': 330}
{'name': 'JJC Code 124', 'val': 329}
{'name': 'JJC Code 125', 'val': 331}
{'name': 'JJC Code 126', 'val': 159}
{'nam

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

### there seems to be some server timeout happening, so resuming the scrape where it left off

In [26]:
for p in list1[139:]:
    print(p)
    payload=f"jjcid={p['val']}"
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
    }
    r2 = s.post(url, headers=headers, data=payload)
    soup2 = BeautifulSoup( r2.text , 'lxml')
    polyHolder = soup2.select_one('script').contents[0].split('var triangleCoords = ')[1].split(';')[0].replace('\n','')
    polyHolder2 = hjson.loads(polyHolder)
    coords=[]
    for item in polyHolder2:
        lat = item['lat']
        lon = item['lng']
        coords.append([lon,lat])
    
    propHolder1 = soup2.select('div')[1].text.split('\n')
    properties = {}
    for l in propHolder1:
        l1 = l.strip()
        if len(l1):
            l2 = l1.split(' : ')
            properties[l2[0]] = l2[1]
    feature = {"type": "Feature",
        "properties": properties,
        "geometry": {
            "type": "Polygon",
            "coordinates": [coords]
        }
    }
    geojson['features'].append(feature)
    time.sleep(1)

{'name': 'JJC Code 143', 'val': 176}
{'name': 'JJC Code 144', 'val': 177}
{'name': 'JJC Code 145', 'val': 178}
{'name': 'JJC Code 146', 'val': 179}
{'name': 'JJC Code 147', 'val': 636}
{'name': 'JJC Code 148', 'val': 635}
{'name': 'JJC Code 149', 'val': 634}
{'name': 'JJC Code 150', 'val': 633}
{'name': 'JJC Code 151', 'val': 632}
{'name': 'JJC Code 152', 'val': 627}
{'name': 'JJC Code 153', 'val': 649}
{'name': 'JJC Code 154', 'val': 650}
{'name': 'JJC Code 155', 'val': 651}
{'name': 'JJC Code 156', 'val': 520}


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [29]:
for p in list1[152:]:
    print(p)
    payload=f"jjcid={p['val']}"
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
    }
    r2 = s.post(url, headers=headers, data=payload)
    soup2 = BeautifulSoup( r2.text , 'lxml')
    polyHolder = soup2.select_one('script').contents[0].split('var triangleCoords = ')[1].split(';')[0].replace('\n','')
    polyHolder2 = hjson.loads(polyHolder)
    coords=[]
    for item in polyHolder2:
        lat = item['lat']
        lon = item['lng']
        coords.append([lon,lat])
    
    propHolder1 = soup2.select('div')[1].text.split('\n')
    properties = {}
    for l in propHolder1:
        l1 = l.strip()
        if len(l1):
            l2 = l1.split(' : ')
            properties[l2[0]] = l2[1]
    feature = {"type": "Feature",
        "properties": properties,
        "geometry": {
            "type": "Polygon",
            "coordinates": [coords]
        }
    }
    geojson['features'].append(feature)
    time.sleep(1)

{'name': 'JJC Code 156', 'val': 520}
{'name': 'JJC Code 157', 'val': 507}
{'name': 'JJC Code 158', 'val': 518}
{'name': 'JJC Code 159', 'val': 519}
{'name': 'JJC Code 160', 'val': 517}
{'name': 'JJC Code 161', 'val': 516}
{'name': 'JJC Code 162', 'val': 501}
{'name': 'JJC Code 163', 'val': 502}
{'name': 'JJC Code 164', 'val': 450}
{'name': 'JJC Code 165', 'val': 482}
{'name': 'JJC Code 166', 'val': 490}
{'name': 'JJC Code 167', 'val': 505}
{'name': 'JJC Code 168', 'val': 433}
{'name': 'JJC Code 169', 'val': 493}
{'name': 'JJC Code 170', 'val': 492}
{'name': 'JJC Code 171', 'val': 494}
{'name': 'JJC Code 172', 'val': 486}
{'name': 'JJC Code 173', 'val': 504}
{'name': 'JJC Code 174', 'val': 503}
{'name': 'JJC Code 175', 'val': 495}
{'name': 'JJC Code 176', 'val': 496}
{'name': 'JJC Code 177', 'val': 498}
{'name': 'JJC Code 178', 'val': 497}
{'name': 'JJC Code 179', 'val': 485}
{'name': 'JJC Code 180', 'val': 484}
{'name': 'JJC Code 181', 'val': 487}
{'name': 'JJC Code 182', 'val': 489}
{

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [30]:
for p in list1[185:]:
    print(p)
    payload=f"jjcid={p['val']}"
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
    }
    try:
        r2 = s.post(url, headers=headers, data=payload)
    except:
        print("stopped at",p)
        break
    soup2 = BeautifulSoup( r2.text , 'lxml')
    polyHolder = soup2.select_one('script').contents[0].split('var triangleCoords = ')[1].split(';')[0].replace('\n','')
    polyHolder2 = hjson.loads(polyHolder)
    coords=[]
    for item in polyHolder2:
        lat = item['lat']
        lon = item['lng']
        coords.append([lon,lat])
    
    propHolder1 = soup2.select('div')[1].text.split('\n')
    properties = {}
    for l in propHolder1:
        l1 = l.strip()
        if len(l1):
            l2 = l1.split(' : ')
            properties[l2[0]] = l2[1]
    feature = {"type": "Feature",
        "properties": properties,
        "geometry": {
            "type": "Polygon",
            "coordinates": [coords]
        }
    }
    geojson['features'].append(feature)
    time.sleep(1)

{'name': 'JJC Code 189', 'val': 666}
{'name': 'JJC Code 190', 'val': 145}
{'name': 'JJC Code 191', 'val': 146}
{'name': 'JJC Code 192', 'val': 147}
{'name': 'JJC Code 193', 'val': 143}
{'name': 'JJC Code 194', 'val': 144}
{'name': 'JJC Code 195', 'val': 320}
{'name': 'JJC Code 196', 'val': 348}
{'name': 'JJC Code 197', 'val': 325}
{'name': 'JJC Code 198', 'val': 349}
{'name': 'JJC Code 199', 'val': 347}
{'name': 'JJC Code 200', 'val': 322}
{'name': 'JJC Code 201', 'val': 321}
{'name': 'JJC Code 202', 'val': 324}
{'name': 'JJC Code 203', 'val': 323}
{'name': 'JJC Code 204', 'val': 413}
{'name': 'JJC Code 205', 'val': 412}
{'name': 'JJC Code 206', 'val': 414}
{'name': 'JJC Code 207', 'val': 415}
{'name': 'JJC Code 208', 'val': 416}
{'name': 'JJC Code 209', 'val': 418}
{'name': 'JJC Code 210', 'val': 417}
{'name': 'JJC Code 211', 'val': 419}
{'name': 'JJC Code 212', 'val': 420}
{'name': 'JJC Code 213', 'val': 421}
{'name': 'JJC Code 214', 'val': 422}
{'name': 'JJC Code 215', 'val': 423}
s

In [33]:
for p in list1[211:]:
    print(p)
    payload=f"jjcid={p['val']}"
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
    }
    try:
        r2 = requests.post(url, headers=headers, data=payload)
    except:
        print("stopped at",p)
        break
    soup2 = BeautifulSoup( r2.text , 'lxml')
    polyHolder = soup2.select_one('script').contents[0].split('var triangleCoords = ')[1].split(';')[0].replace('\n','')
    polyHolder2 = hjson.loads(polyHolder)
    coords=[]
    for item in polyHolder2:
        lat = item['lat']
        lon = item['lng']
        coords.append([lon,lat])
    
    propHolder1 = soup2.select('div')[1].text.split('\n')
    properties = {}
    for l in propHolder1:
        l1 = l.strip()
        if len(l1):
            l2 = l1.split(' : ')
            properties[l2[0]] = l2[1]
    feature = {"type": "Feature",
        "properties": properties,
        "geometry": {
            "type": "Polygon",
            "coordinates": [coords]
        }
    }
    geojson['features'].append(feature)
    time.sleep(1)

{'name': 'JJC Code 215', 'val': 423}
{'name': 'JJC Code 216', 'val': 424}
{'name': 'JJC Code 217', 'val': 425}
{'name': 'JJC Code 218', 'val': 74}
{'name': 'JJC Code 219', 'val': 72}
{'name': 'JJC Code 220', 'val': 73}
{'name': 'JJC Code 221', 'val': 71}
{'name': 'JJC Code 222', 'val': 60}
{'name': 'JJC Code 223', 'val': 61}
{'name': 'JJC Code 224', 'val': 59}
{'name': 'JJC Code 225', 'val': 58}
{'name': 'JJC Code 226', 'val': 62}
{'name': 'JJC Code 227', 'val': 64}
{'name': 'JJC Code 228', 'val': 63}
{'name': 'JJC Code 229', 'val': 67}
{'name': 'JJC Code 230', 'val': 68}
{'name': 'JJC Code 231', 'val': 66}
{'name': 'JJC Code 232', 'val': 65}
{'name': 'JJC Code 233', 'val': 253}
{'name': 'JJC Code 234', 'val': 255}
{'name': 'JJC Code 235', 'val': 263}
{'name': 'JJC Code 236', 'val': 262}
{'name': 'JJC Code 237', 'val': 254}
{'name': 'JJC Code 238', 'val': 250}
{'name': 'JJC Code 239', 'val': 252}
{'name': 'JJC Code 240', 'val': 257}
{'name': 'JJC Code 241', 'val': 256}
{'name': 'JJC Co

In [34]:
def fetchShape(p):
    payload=f"jjcid={p['val']}"
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
    }
    try:
        r2 = requests.post(url, headers=headers, data=payload)
    except:
        print("stopped at",p)
        return False
    soup2 = BeautifulSoup( r2.text , 'lxml')
    polyHolder = soup2.select_one('script').contents[0].split('var triangleCoords = ')[1].split(';')[0].replace('\n','')
    polyHolder2 = hjson.loads(polyHolder)
    coords=[]
    for item in polyHolder2:
        lat = item['lat']
        lon = item['lng']
        coords.append([lon,lat])
    
    propHolder1 = soup2.select('div')[1].text.split('\n')
    properties = {}
    for l in propHolder1:
        l1 = l.strip()
        if len(l1):
            l2 = l1.split(' : ')
            properties[l2[0]] = l2[1]
    feature = {"type": "Feature",
        "properties": properties,
        "geometry": {
            "type": "Polygon",
            "coordinates": [coords]
        }
    }

In [38]:
for p in list1[240:]:
    print(p)
    feature = False
    while(not feature):
        feature = fetchShape(p)
        if not feature:
            timeout = 61
            print(f"Timeout, waiting {timeout} secs")
            time.sleep(timeout)
        else:
            geojson['features'].append(feature)
            time.sleep(1)
            

{'name': 'JJC Code 244', 'val': 261}
Timeout, waiting 61 secs


KeyboardInterrupt: 

In [ ]:
### Seems stuck here.

In [40]:
# geojson

In [39]:
json.dump(geojson, open('jj.geojson','w'), indent=2)